# Effective Kraus Rank

In [2]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Real Hardware

In [3]:
n = 3
d = 2**n

In [4]:
model1 =fit_model(channel=KrausMap(d=d, 
                                   rank=64),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  N_map=500,
                  N_spam=None,
                  num_iter_map=2000,
                  num_iter_spam=2000,
                  filename="belem_concatenate_4layer0",
                  verbose=True
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0029162906585489977


  0%|          | 0/4000 [00:00<?, ?it/s]

In [5]:
model2 =fit_model(channel=KrausMap(d=d, 
                                   rank=64),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  N_map=500,
                  N_spam=None,
                  num_iter_map=2000,
                  num_iter_spam=2000,
                  filename="belem_concatenate_4layer1",
                  verbose=True
                  )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0029277294279222233


  0%|          | 0/2000 [00:00<?, ?it/s]

In [6]:
model_full = fit_model(channel=KrausMap(d=d, 
                                        rank=64),
                       spam=SPAM(init = InitialState(d),
                                 povm = CorruptionMatrix(d),
                                 optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                ),
                      N_map=500,
                      N_spam=None,
                      num_iter_map=2000,
                      num_iter_spam=2000,
                      filename="belem_concatenate_4layer2",
                      verbose = True,
                  )

  0%|          | 0/4000 [00:00<?, ?it/s]

0.002867661830198437


  0%|          | 0/4000 [00:00<?, ?it/s]

In [19]:
channel1 = model1.channel
channel2 = model2.channel
channel_concat = channel_to_choi_map([channel1, channel2])
channel_full = model_full.channel

In [21]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target1 = pqc_basic(n, 4).reverse_bits()
circuit_target2 = pqc_basic(n, 4).reverse_bits()

circuit_target12 = deepcopy(circuit_target1)
circuit_target12 = circuit_target12.compose(circuit_target2)
circuit_target_list = [circuit_target1, circuit_target2, circuit_target12]

In [26]:
U1 = Operator(circuit_target1).data
U2 = Operator(circuit_target2).data
U_full = Operator(circuit_target12).data

In [29]:
channel_ideal1 = DilutedKrausMap(U1, c=0.999, d=d, rank=1)
channel_ideal2 = DilutedKrausMap(U2, c=0.999, d=d, rank=1)
channel_ideal_full = DilutedKrausMap(U_full, c=0.999, d=d, rank=1)

In [38]:
print(f" U1 vs T1: {channel_fidelity(channel_ideal1, channel1):.4f}")
print(f" U2 vs T2: {channel_fidelity(channel_ideal2, channel2):.4f}")

print(f" U Full vs T Full: {channel_fidelity(channel_ideal_full, channel_full):.4f}")
print(f" U Full vs T2*T1: {channel_fidelity(channel_ideal_full, channel_concat):.4f}")
      
print(f" T Full vs T2*T1: {channel_fidelity(channel_full, channel_concat):.4f}")

 U1 vs T1: 0.5198
 U2 vs T2: 0.5677
 U Full vs T Full: 0.4361
 U Full vs T2*T1: 0.3049
 T Full vs T2*T1: 0.6732


In [32]:
model1 = fit_model(channel=KrausMap(d=d, 
                                    rank=64),
                   spam=SPAM(init = InitialState(d),
                             povm = CorruptionMatrix(d),
                             optimizer = tf.optimizers.Adam(learning_rate=0.01),
                             ),
                   N_map=500,
                   N_spam=None,
                   num_iter_map=2000,
                   num_iter_spam=2000,
                   filename="belem_concatenate_8layer0",
                   verbose=True
                   )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.002837134406602385


  0%|          | 0/2000 [00:00<?, ?it/s]

In [33]:
model2 = fit_model(channel=KrausMap(d=d, 
                                    rank=64),
                   spam=SPAM(init = InitialState(d),
                             povm = CorruptionMatrix(d),
                             optimizer = tf.optimizers.Adam(learning_rate=0.01),
                             ),
                   N_map=500,
                   N_spam=None,
                   num_iter_map=2000,
                   num_iter_spam=2000,
                   filename="belem_concatenate_8layer1",
                   verbose=True
                   )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0029792934991362437


  0%|          | 0/2000 [00:00<?, ?it/s]

In [34]:
model3 = fit_model(channel=KrausMap(d=d, 
                                    rank=64),
                   spam=SPAM(init = InitialState(d),
                             povm = CorruptionMatrix(d),
                             optimizer = tf.optimizers.Adam(learning_rate=0.01),
                             ),
                   N_map=500,
                   N_spam=None,
                   num_iter_map=2000,
                   num_iter_spam=2000,
                   filename="belem_concatenate_8layer2",
                   verbose=True
                   )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.003157538763345682


  0%|          | 0/2000 [00:00<?, ?it/s]

In [43]:
channel1 = model1.channel
channel2 = model2.channel
channel_concat = channel_to_choi_map([channel1, channel2])
channel_full = model3.channel

In [44]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target1 = pqc_basic(n, 8).reverse_bits()
circuit_target2 = pqc_basic(n, 8).reverse_bits()

circuit_target12 = deepcopy(circuit_target1)
circuit_target12 = circuit_target12.compose(circuit_target2)
circuit_target_list = [circuit_target1, circuit_target2, circuit_target12]

In [45]:
U1 = Operator(circuit_target1).data
U2 = Operator(circuit_target2).data
U_full = Operator(circuit_target12).data

channel_ideal1 = DilutedKrausMap(U1, c=0.999, d=d, rank=1)
channel_ideal2 = DilutedKrausMap(U2, c=0.999, d=d, rank=1)
channel_ideal_full = DilutedKrausMap(U_full, c=0.999, d=d, rank=1)

In [46]:
print(f" U1 vs T1: {channel_fidelity(channel_ideal1, channel1):.4f}")
print(f" U2 vs T2: {channel_fidelity(channel_ideal2, channel2):.4f}")

print(f" U Full vs T Full: {channel_fidelity(channel_ideal_full, channel_full):.4f}")
print(f" U Full vs T2*T1: {channel_fidelity(channel_ideal_full, channel_concat):.4f}")
      
print(f" T Full vs T2*T1: {channel_fidelity(channel_full, channel_concat):.4f}")

 U1 vs T1: 0.4301
 U2 vs T2: 0.4607
 U Full vs T Full: 0.2764
 U Full vs T2*T1: 0.2006
 T Full vs T2*T1: 0.6995


In [48]:
model1 = fit_model(channel=KrausMap(d=d, 
                                    rank=64),
                   spam=SPAM(init = InitialState(d),
                             povm = CorruptionMatrix(d),
                             optimizer = tf.optimizers.Adam(learning_rate=0.01),
                             ),
                   N_map=500,
                   N_spam=None,
                   num_iter_map=2000,
                   num_iter_spam=2000,
                   filename="belem_concatenate_12layer0",
                   verbose=True
                   )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0029895005191711474


  0%|          | 0/2000 [00:00<?, ?it/s]

In [49]:
model2 = fit_model(channel=KrausMap(d=d, 
                                    rank=64),
                   spam=SPAM(init = InitialState(d),
                             povm = CorruptionMatrix(d),
                             optimizer = tf.optimizers.Adam(learning_rate=0.01),
                             ),
                   N_map=500,
                   N_spam=None,
                   num_iter_map=2000,
                   num_iter_spam=2000,
                   filename="belem_concatenate_12layer1",
                   verbose=True
                   )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.003056282783368792


  0%|          | 0/2000 [00:00<?, ?it/s]

In [50]:
model3 = fit_model(channel=KrausMap(d=d, 
                                    rank=64),
                   spam=SPAM(init = InitialState(d),
                             povm = CorruptionMatrix(d),
                             optimizer = tf.optimizers.Adam(learning_rate=0.01),
                             ),
                   N_map=500,
                   N_spam=None,
                   num_iter_map=2000,
                   num_iter_spam=2000,
                   filename="belem_concatenate_12layer2",
                   verbose=True
                   )

  0%|          | 0/2000 [00:00<?, ?it/s]

0.003067306695876659


  0%|          | 0/2000 [00:00<?, ?it/s]

In [54]:
channel1 = model1.channel
channel2 = model2.channel
channel_concat = channel_to_choi_map([channel1, channel2])
channel_full = model3.channel

In [55]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target1 = pqc_basic(n, 12).reverse_bits()
circuit_target2 = pqc_basic(n, 12).reverse_bits()

circuit_target12 = deepcopy(circuit_target1)
circuit_target12 = circuit_target12.compose(circuit_target2)
circuit_target_list = [circuit_target1, circuit_target2, circuit_target12]

In [56]:
U1 = Operator(circuit_target1).data
U2 = Operator(circuit_target2).data
U_full = Operator(circuit_target12).data

channel_ideal1 = DilutedKrausMap(U1, c=0.999, d=d, rank=1)
channel_ideal2 = DilutedKrausMap(U2, c=0.999, d=d, rank=1)
channel_ideal_full = DilutedKrausMap(U_full, c=0.999, d=d, rank=1)

In [57]:
print(f" U1 vs T1: {channel_fidelity(channel_ideal1, channel1):.4f}")
print(f" U2 vs T2: {channel_fidelity(channel_ideal2, channel2):.4f}")

print(f" U Full vs T Full: {channel_fidelity(channel_ideal_full, channel_full):.4f}")
print(f" U Full vs T2*T1: {channel_fidelity(channel_ideal_full, channel_concat):.4f}")
      
print(f" T Full vs T2*T1: {channel_fidelity(channel_full, channel_concat):.4f}")

 U1 vs T1: 0.3538
 U2 vs T2: 0.3046
 U Full vs T Full: 0.1634
 U Full vs T2*T1: 0.1138
 T Full vs T2*T1: 0.7215
